In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3 # 최신 버전으로 설치하면 "Input: must be Tensor, not str" 라는 에러 발생
!pip install torch

     |████████████████████████████████| 46.9MB 66kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 348kB 18.2MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595692 sha256=9c43833063b4789420f31755ce65a71c48bfe4a8c48e0b78180b1f71c63ab556
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 1.2MB 21.5MB/s 
     |████████████████████████████████| 757kB 13.3MB/s 
     |████████████████████████████████| 870kB 55.8MB/s 
     |████████████████████████████████| 3.0MB 53.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=9eec5a0022214a12283a00c2a0e088f805e0875ee81ac942cf67268886159199
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f44

In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-hdd9tlug
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-hdd9tlug
  Created wheel for kobert: filename=kobert-0.1.2-cp37-none-any.whl size=12708 sha256=79574fe1b216d7c73e99c793ba6715bb50542318d756c7edc6a83c62510424c0
  Stored in directory: /tmp/pip-ephem-wheel-cache-le2_lo1f/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [59]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [60]:
device = torch.device("cuda:0")

In [61]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [62]:
import pandas as pd

DF = pd.read_excel('sentence_tokenized.xlsx')
DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39080 entries, 0 to 39079
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  39080 non-null  int64 
 1   sentence    39080 non-null  object
 2   emotion     39080 non-null  object
 3   tokenized   39080 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.2+ MB


In [63]:
DF.head()

,Unnamed: 0,sentence,emotion,tokenized
0,0,게임방 가서 사광 클릭해서 대기 4600명 뚫고 성공,행복,"['게임방', '가', '아서', '사광', '클릭', '하', '여서', '대기'..."
1,1,신곡 지붕킥 하자,행복,"['신곡', '지붕킥', '하', '자']"
2,2,VR로 즐길 듯 기대되네,행복,"['VR', '로', '즐기', 'ㄹ', '듯', '기대', '되', '네']"
3,3,뒷북 꿀 잼,행복,"['뒷북', '꿀', '잼']"
4,4,굿 꿩 먹고 알 먹고 누이 좋고 매부 좋고,행복,"['굿', '꿩', '먹', '고', '알', '먹', '고', '누이', '좋',..."


In [64]:
dataset_train1 = pd.DataFrame()

In [65]:
dataset_train1['sentence'] = DF['sentence']
dataset_train1['emotion'] = DF['emotion']

In [66]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(dataset_train1['emotion'])
dataset_train1['emotion'] = encoder.transform(dataset_train1['emotion'])

dataset_train1.head()

,sentence,emotion
0,게임방 가서 사광 클릭해서 대기 4600명 뚫고 성공,5
1,신곡 지붕킥 하자,5
2,VR로 즐길 듯 기대되네,5
3,뒷북 꿀 잼,5
4,굿 꿩 먹고 알 먹고 누이 좋고 매부 좋고,5


In [67]:
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))

mapping

{0: '공포', 1: '놀람', 2: '분노', 3: '슬픔', 4: '중립', 5: '행복', 6: '혐오'}

In [68]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(dataset_train1, test_size=0.2, random_state=2045)

train

,sentence,emotion
25368,인간 세상 살면서 어쩌면 잊힌다는 기억이 가장 슬픈 것일 거라는 생각이 들었네요,3
23314,남자친구한테 기대지 않으려면 어떻게 해야 할까요,0
18472,재석 빠ㅡㅡ 강호동에게 맞을 때마다 가슴이 아픕니다ㅡㅡ쿵,3
2576,깨끗한 연기로 우리에게 큰 기쁨을 주네요,5
21492,좀 무섭기도 하고 진짠가요ㅜㅜ,0
...,...,...
31463,달은 멀어서 가기 힘들어요,4
14119,사랑 웃기고 자빠졌네,2
29721,도미니카 가서 박현준이랑 원투 펀치 가게인데,4
36361,댓글 부대 박정원 돈 사모들 드글드글,6


In [69]:
for i in train.values:
  print(i[0])
  break

인간 세상 살면서 어쩌면 잊힌다는 기억이 가장 슬픈 것일 거라는 생각이 들었네요


In [70]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [71]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [72]:
max_len = 80
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [73]:
data_train = BERTDataset(train.values, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(test.values, 0, 1, tok, max_len, True, False)

In [74]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [75]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 7, # softmax 사용 <- binary일 경우는 2
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [76]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [77]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [78]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [79]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [80]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [81]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [82]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 2.0425102710723877 train acc 0.140625
epoch 1 batch id 201 loss 1.6535608768463135 train acc 0.25777363184079605
epoch 1 batch id 401 loss 1.4179487228393555 train acc 0.37535068578553615

epoch 1 train acc 0.4061221881390593


epoch 2 batch id 1 loss 1.18624746799469 train acc 0.59375
epoch 2 batch id 201 loss 1.4374593496322632 train acc 0.5551150497512438
epoch 2 batch id 401 loss 1.217072606086731 train acc 0.5889962593516209

epoch 2 train acc 0.601482617586912


epoch 3 batch id 1 loss 0.9982396364212036 train acc 0.6875
epoch 3 batch id 201 loss 1.1668246984481812 train acc 0.6754508706467661
epoch 3 batch id 401 loss 0.9692827463150024 train acc 0.7068266832917706

epoch 3 train acc 0.7188139059304703


epoch 4 batch id 1 loss 0.7085514664649963 train acc 0.796875
epoch 4 batch id 201 loss 0.8588617444038391 train acc 0.7706778606965174
epoch 4 batch id 401 loss 0.7901065349578857 train acc 0.7945370947630923

epoch 4 train acc 0.8030099693251533


epoch 5 batch id 1 loss 0.6603320240974426 train acc 0.78125
epoch 5 batch id 201 loss 0.6366026997566223 train acc 0.8357431592039801
epoch 5 batch id 401 loss 0.5637593865394592 train acc 0.8472568578553616

epoch 5 train acc 0.8521855828220859


In [103]:
test_sentence = '나 시험 망했어'
test_label = 3

In [104]:
unseen_test = pd.DataFrame([[test_sentence, test_label]], columns = [['sentence', 'emotion']])
unseen_values = unseen_test.values

In [105]:
unseen_values

array([['나 시험 망했어', 3]], dtype=object)

In [106]:
test_set = BERTDataset(unseen_values, 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [107]:
test_acc = 0

In [109]:
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  out = model(token_ids, valid_length, segment_ids)
  print(out)
    
  model.eval() # 평가 모드로 변경
    
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
      token_ids = token_ids.long().to(device)
      segment_ids = segment_ids.long().to(device)
      valid_length= valid_length
      label = label.long().to(device)
      out = model(token_ids, valid_length, segment_ids)
      test_acc += calc_accuracy(out, label)

print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


RuntimeError: ignored